In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

Домашнее задание 3: Метод К-средних

## 1. Выберите 3-6 количественных признаков, объясните выбор и примените метод К-средних:

Мы выберем 'Confirmed', 'Deaths', 'Recovered', 'Active', и 'New cases', как это было сделано в предыдущем задании. Эти признаки представляют различные аспекты пандемии COVID-19 и могут дать интересные результаты при кластеризации.

Стандартизация данных перед кластеризацией помогает уравнять вклад каждого признака, несмотря на различия в их масштабах.

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Загрузка данных
df = pd.read_csv('country_wise_latest.csv')

# Выбор признаков для кластеризации
features_for_clustering = ['Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases']

# Подготовка данных для кластеризации
clustering_data = df[features_for_clustering]

# Стандартизация данных перед кластеризацией
scaler = StandardScaler()
data_scaled = scaler.fit_transform(clustering_data)

# Вывод первых нескольких строк стандартизированных данных для проверки
pd.DataFrame(data_scaled, columns=features_for_clustering).head()


,Confirmed,Deaths,Recovered,Active,New cases
0,-0.135676,-0.158475,-0.134087,-0.113774,-0.196126
1,-0.217768,-0.238477,-0.252461,-0.150459,-0.194195
2,-0.157361,-0.166013,-0.167623,-0.122342,-0.106576
3,-0.228160,-0.245019,-0.262699,-0.159573,-0.212983
4,-0.228048,-0.245801,-0.265657,-0.156682,-0.211578


Применение алгоритма K-средних с K=5 и K=9, выполняя 10 случайных инициализаций для каждого значения K, чтобы выбрать наилучшее разбиение.

Тут 10 случайных инициализаций


In [ ]:
# Функция для выполнения кластеризации K-средних с различным числом кластеров и случайными инициализациями
def perform_kmeans(n_clusters, data, n_init=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=n_init, random_state=0)
    kmeans.fit(data)
    return kmeans

# Выполнение K-средних для K=5
kmeans_5 = perform_kmeans(5, data_scaled)

# Выполнение K-средних для K=9
kmeans_9 = perform_kmeans(9, data_scaled)

# Сохранение результатов кластеризации и инерции для каждого случая
clusters_5 = kmeans_5.labels_
inertia_5 = kmeans_5.inertia_

clusters_9 = kmeans_9.labels_
inertia_9 = kmeans_9.inertia_

inertia_5, inertia_9

(44.04053426458702, 11.100064928568965)

Значения (44.04053426458702, 11.100064928568965), которые вывелись в результате выполнения вашего кода, представляют собой значения инерции для двух различных кластеризаций методом K-средних (K-Means), выполненных на вашем наборе данных.


Мы будем оценивать качество кластеризации по сумме квадратов расстояний от точек до центров их кластеров (inertia).

___

Что такое Инерция в Контексте K-средних - Инерция является мерой внутрикластерного расстояния в алгоритме K-средних. Она рассчитывается как сумма квадратов расстояний каждой точки данных до центра ее кластера. Низкая инерция указывает на то, что точки данных находятся ближе к центрам своих кластеров, что обычно считается признаком хорошей кластеризации.

Инерция для K=5: 44.04053426458702. Это значение отражает сумму квадратов расстояний точек до центров их кластеров. Меньшее значение инерции указывает на более плотные кластеры.

Инерция для K=9: 11.100064928568965. Ожидаемо, что с увеличением числа кластеров инерция уменьшается, так как точки становятся ближе к центрам своих кластеров.

___

Выберите то, которое доставляет минимум критерию метода -
---



In [ ]:
def perform_kmeans_optimized(n_clusters, data, n_init=10):
    best_inertia = np.inf
    best_kmeans = None
    for _ in range(n_init):
        kmeans = KMeans(n_clusters=n_clusters, n_init=1, random_state=np.random.randint(10000))
        kmeans.fit(data)
        if kmeans.inertia_ < best_inertia:
            best_inertia = kmeans.inertia_
            best_kmeans = kmeans
    return best_kmeans, best_inertia

# Применение функции для K=5 и K=9
kmeans_optimized_5, inertia_5 = perform_kmeans_optimized(5, data_scaled)
kmeans_optimized_9, inertia_9 = perform_kmeans_optimized(9, data_scaled)

# Вывод моделей и их инерции
print("K=5: Model:", kmeans_optimized_5, "Inertia:", inertia_5)
print("K=9: Model:", kmeans_optimized_9, "Inertia:", inertia_9)


K=5: Model: KMeans(n_clusters=5, n_init=1, random_state=3428) Inertia: 44.04053426458702
K=9: Model: KMeans(n_clusters=9, n_init=1, random_state=4396) Inertia: 11.100064928568965


Для каждого значения K (5 и 9) была выбрана лучшая инициализация из 10 случайных инициализаций, исходя из критерия минимальной инерции

## 2. Интерпретация полученных разбиений. Сравнение разбиений для K=5 и K=9 с точки зрения интерпретации

Интерпретация внутрикластерных средних значений по сравнению с общими средними (которые равны 0 для каждого признака из-за стандартизации):

Для K=5:

Кластер 1 и 2 выделяются с высокими средними значениями по всем признакам, что может указывать на страны с наиболее серьезной ситуацией по COVID-19.
Кластер 3 также имеет высокие значения, но несколько ниже, чем в кластерах 1 и 2.
Кластеры 0 и 4 имеют отрицательные или низкие положительные средние значения, что может соответствовать странам с менее серьезной ситуацией.
Для K=9:

Кластеры 1, 2 и 3 аналогичны предыдущему случаю с K=5, выделяя страны с высокими значениями по всем признакам.
Кластеры 4, 5, 6, 7 и 8 показывают более дифференцированные группы стран с различными уровнями влияния COVID-19.


In [ ]:
# Функция для расчета средних значений внутри кластеров и их сравнения с общими средними
def compare_cluster_means(data_scaled, clusters, features):
    df_scaled = pd.DataFrame(data_scaled, columns=features)
    df_scaled['Cluster'] = clusters

    # Расчет средних значений для каждого кластера
    cluster_means = df_scaled.groupby('Cluster').mean()

    # Расчет общих средних значений для всех данных
    overall_means = df_scaled[features].mean()

    return cluster_means, overall_means

# Расчет средних значений для кластеров K=5
cluster_means_5, overall_means_5 = compare_cluster_means(data_scaled, clusters_5, features_for_clustering)

# Расчет средних значений для кластеров K=9
cluster_means_9, overall_means_9 = compare_cluster_means(data_scaled, clusters_9, features_for_clustering)

cluster_means_5, cluster_means_9


(         Confirmed     Deaths  Recovered     Active  New cases
 Cluster                                                       
 0        -0.165909  -0.195822  -0.183034  -0.121992  -0.165103
 1        10.991922  10.276696   6.722793  13.078150   9.677298
 2         6.158229   5.982007   9.468680   2.228451   3.873698
 3         3.641041   2.127005   4.747677   2.169148   7.591465
 4         0.719195   1.408311   0.975074   0.329900   0.674576,
          Confirmed     Deaths  Recovered     Active  New cases
 Cluster                                                       
 0        -0.207000  -0.233408  -0.234212  -0.147715  -0.195093
 1         6.158229   5.982007   9.468680   2.228451   3.873698
 2        10.991922  10.276696   6.722793  13.078150   9.677298
 3         3.641041   2.127005   4.747677   2.169148   7.591465
 4         0.520859   2.362842   0.508512   0.326381   0.079550
 5         0.615440   0.718234   0.797255   0.347609   2.424554
 6         0.543524   0.315336   1.0245

In [ ]:
cluster_means_5

,Confirmed,Deaths,Recovered,Active,New cases
Cluster,,,,,
0,-0.165909,-0.195822,-0.183034,-0.121992,-0.165103
1,10.991922,10.276696,6.722793,13.078150,9.677298
2,6.158229,5.982007,9.468680,2.228451,3.873698
3,3.641041,2.127005,4.747677,2.169148,7.591465
4,0.719195,1.408311,0.975074,0.329900,0.674576


Средние Значения в Кластерах для K=5:

Таблица cluster_means_5 показывает средние значения признаков ('Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases') в каждом из 5 кластеров после стандартизации данных.

Каждая строка таблицы соответствует одному кластеру. Значения в столбцах отражают средний уровень каждого признака в кластере.

Например, в кластере 1 значения по всем признакам значительно выше средних, что может указывать на страны с высоким уровнем распространения COVID-19.

In [ ]:
cluster_means_9

,Confirmed,Deaths,Recovered,Active,New cases
Cluster,,,,,
0,-0.207000,-0.233408,-0.234212,-0.147715,-0.195093
1,6.158229,5.982007,9.468680,2.228451,3.873698
2,10.991922,10.276696,6.722793,13.078150,9.677298
3,3.641041,2.127005,4.747677,2.169148,7.591465
4,0.520859,2.362842,0.508512,0.326381,0.079550
5,0.615440,0.718234,0.797255,0.347609,2.424554
6,0.543524,0.315336,1.024549,0.042374,0.191526
7,1.905738,0.699495,2.908164,0.785387,0.769794
8,0.004192,-0.010555,-0.033050,0.037694,0.008133


Средние Значения в Кластерах для K=9:

Таблица cluster_means_9 показывает аналогичные средние значения, но для разбиения на 9 кластеров.

Большее количество кластеров позволяет увидеть более детализированное разделение данных, что может привести к более точной сегментации стран по характеристикам распространения вируса.

In [ ]:
overall_means_5

Confirmed   -9.499234e-18
Deaths       5.937022e-18
Recovered    2.374809e-17
Active       4.749617e-18
New cases   -4.749617e-18
dtype: float64

In [ ]:
overall_means_9

Confirmed   -9.499234e-18
Deaths       5.937022e-18
Recovered    2.374809e-17
Active       4.749617e-18
New cases   -4.749617e-18
dtype: float64

Общие Средние Значения:

overall_means_5 и overall_means_9 представляют средние значения по всем странам (до кластеризации) для каждого из признаков. Эти значения одинаковы для обоих разбиений, так как они основаны на исходных данных до кластеризации.

Сравнение разбиений для K=5 и K=9 с точки зрения интерпретации

Разбиение на K=5 предоставляет более обобщенное представление о распределении стран по уровню воздействия COVID-19. Это разбиение может быть полезным для анализа на высоком уровне, выделяя группы стран с высоким, средним и низким уровнем воздействия пандемии.

Разбиение на K=9 дает более детальное представление, разделяя страны на более мелкие группы. Это может быть полезно для более точного анализа различий между странами и для выявления уникальных паттернов распространения заболевания.

С точки зрения интерпретации, оба разбиения имеют свои преимущества. Разбиение на K=5 удобно для получения общего представления о распределении стран, тогда как K=9 лучше подходит для детализированного анализа и выявления более тонких отличий между странами. Выбор оптимального числа кластеров зависит от целей анализа и доступных ресурсов для интерпретации результатов. ​